In [4]:
from model.MF import *
from preprocess.Gowalla import *
from evaluation.MF_SISA import *
pd.options.display.max_rows = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = './dataset/Gowalla'
dataset = Gowalla(path)

# Data(num_nodes=144242, edge_index=[2, 2380730], edge_label_index=[2, 603378])
data = dataset.get()
num_users, num_books = dataset.getNumber()
config = {
    'k': 20,
    'lr': 1e-5,  # over-fitting
    'epochs': 100,
    'num_layers': 2,
    'batch_size': 8192,
    'embedding_dim': 64,
    'num_users': num_users,
    'num_books': num_books,
    'tuning_type': None,
    "weight_decay": 1e-7,
    'global_bias':(data.edge_index.size(1) + data.edge_label_index.size(1) + 2) / (num_books * num_users)
}
model = MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device)

In [5]:
config['epochs'] = 100
config['num_shards'] = 5
model_list =[]

for _ in range(config['num_shards']):
    model_list.append(MF(
    num_users= config['num_users'],
    num_items= config['num_books'],
    mean = config['global_bias'],
    embedding_dim = config['embedding_dim']
).to(device))

shard_models, shards, epoch_tracks, test_topks = sisa_MF_eva(model_list, config, data, device)

Epoch: 001, Loss: 0.9983, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 002, Loss: 0.9978, HR@20: 0.0002, Recall@20: 0.0004, NDCG@20: 0.0003
Epoch: 003, Loss: 0.9973, HR@20: 0.0002, Recall@20: 0.0004, NDCG@20: 0.0003
Epoch: 004, Loss: 0.9969, HR@20: 0.0002, Recall@20: 0.0004, NDCG@20: 0.0003
Epoch: 005, Loss: 0.9964, HR@20: 0.0002, Recall@20: 0.0004, NDCG@20: 0.0003
Epoch: 006, Loss: 0.9959, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 007, Loss: 0.9955, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 008, Loss: 0.9950, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 009, Loss: 0.9945, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 010, Loss: 0.9941, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 011, Loss: 0.9936, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 012, Loss: 0.9931, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 013, Loss: 0.9927, HR@20: 0.0002, Recall@20: 0.0005, NDCG@20: 0.0003
Epoch: 014, 

In [ ]:
# Basic setting
num_edges = data.edge_index.size(1)
perm = torch.randperm(num_edges)
split = int(num_edges * 0.1)  # 10% of edges will be retained

# Define the data for forget and retain dataset
forget_data = Data()
retain_data = Data()

forget_data.num_nodes = data.num_nodes
retain_data.num_nodes = data.num_nodes
forget_data.edge_index = data.edge_index[:, perm[:split]]
retain_data.edge_index = data.edge_index[:, perm[split:]]
forget_data.edge_label_index = data.edge_label_index
retain_data.edge_label_index = data.edge_label_index

In [ ]:
shard_models = sisa_MF_unlearning_eva(shard_models, shards, retain_data, forget_data, config, device)

In [ ]:
sisa_MF_forget_data_eva(shard_models, forget_data, config, device)